# Aggregate proteomic data across studies

For best results, first run the [IdentifierUpdates_Proteomics](../curation/IdentifierUpdates_Proteomics.ipynb) notebook first to remove old/obsolete accessions and map to current UniProt database. 
## Load protein table

Run the  notebook first to create the protein table.
Note: Requires internet connection to download information from the UniProt.

## Setup
### Import packages

In [ ]:
from warnings import warn

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from rbc_gem_utils import check_database_release_online, get_dirpath, show_versions
from rbc_gem_utils.database.uniprot import UNIPROT_DB_TAG, query_UniProt

# Display versions of last time notebook ran and worked
show_versions()
plt.rcParams["font.family"] = "Arial"

## Check UniProt release
If the release does not match the expected release, it is because database has been updated since the last time this code was utilized. 

* According to [UniProt](https://www.uniprot.org/help/downloads), updates to the database are made every eight weeks.
* If the current release does not match the expected release, it is because database has been updated since the last time this code was utilized.
    * If the notebook works without needing any significant modifications, the only update needed is to the release in the [uniprot.py](../../src/rbc_gem_utils/database/uniprot.py) source code file to resolve the issue.

In [ ]:
use_interim = not check_database_release_online(UNIPROT_DB_TAG, verbose=True, **{})
# Use different directory paths for unexpected behavior
if use_interim:
    warn(
        "Online release of database has been updated since the last time notebook was used."
    )

excel_filepath_processed = (
    get_dirpath("proteomics", use_temp="external") / "proteomics_aggregated.xlsx"
)
overwrite = True
results_path = get_dirpath(use_temp="processed") / "AllProteomicEvidence"
results_path.mkdir(exist_ok=True, parents=True)

### Load table of contents

In [ ]:
df_contents = pd.read_excel(
    excel_filepath_processed, sheet_name="Table of Contents", dtype=str
).fillna("")
df_contents

### Aggregate proteins into table

In [ ]:
all_sheet_dfs = {"Table of Contents": df_contents}

index_name = "Uniprot"
df_proteins_aggregated = pd.DataFrame([], columns=[index_name])
for _, (sheet_name, id_type) in df_contents[
    ["PubMed/Sheet Name", "ID type"]
].iterrows():
    if id_type != "UniProt":
        warn("Can only utilize UniProt IDs for final aggregation")
        continue
    df = pd.read_excel(excel_filepath_processed, sheet_name=sheet_name, usecols=[0])
    df.columns = [sheet_name]
    df[index_name] = df[sheet_name]
    all_sheet_dfs[sheet_name] = df.copy()
    df_proteins_aggregated = df_proteins_aggregated.merge(
        df, left_on=index_name, right_on=index_name, how="outer"
    )
df_proteins_aggregated = (
    df_proteins_aggregated.drop_duplicates().set_index(index_name).notna()
)
df_proteins_aggregated.columns.name = "pubmed"
# MIRIAM compliant
df_proteins_aggregated = df_proteins_aggregated.rename(
    {col: f"pubmed:{col}" for col in df_proteins_aggregated.columns}, axis=1
)
df_unreviewed_proteins = df_proteins_aggregated.copy()
df_unreviewed_proteins

### Query UniProt, keep reviewed proteins

In [ ]:
# Extract all relevant information for now and save
query_parameters = {
    "query": " && ".join(
        [
            "(reviewed:true)",
            "(organism_id:9606)",  # Homo sapiens (Human)
        ]
    ),
    "format": "tsv",
    "size": 500,
    "compressed": True,
    "fields": ",".join(
        [
            "reviewed",
            "accession",
            "gene_primary",
        ]
    ),
}

df_results, uniparc, failed_ids, obselete_counts = query_UniProt(
    list(df_unreviewed_proteins.index),
    query_parameters=query_parameters,
    from_db="UniProtKB",
    to_db="UniProtKB",
    return_failed=True,
)

df_results = df_results[df_results["Reviewed"] == "reviewed"].drop_duplicates()
df_results

In [ ]:
df_reviewed_proteins = df_unreviewed_proteins.loc[df_results["Entry"].values]
df_reviewed_proteins = df_reviewed_proteins.astype(int)
df_reviewed_proteins = df_reviewed_proteins.loc[
    df_reviewed_proteins.sum(axis=1).sort_values(ascending=False).index.values
]
df_reviewed_proteins

### Export protein table

In [ ]:
if overwrite:
    df_reviewed_proteins.to_csv(
        get_dirpath("proteomics", use_temp="external") / "proteomic_evidence_table.tsv",
        sep="\t",
        index=True,
    )

## Visualize Protein Table

In [ ]:
# Turned into an xlsx where keys are sheet names and values are DataFrames representing the sheet
all_figure_data = {}
imagetype = "png"
transparent = True

### Create proteomic evidence table for figure
#### Sorted by publication year (original)

In [ ]:
df_proteins = pd.read_csv(
    get_dirpath("proteomics", use_temp="external") / "proteomic_evidence_table.tsv",
    sep="\t",
    index_col=0,
)
df_proteins.columns.name = "pubmed"

# Order by number of times a single  protein was detected across studies
# protein_ids_ordered = df_proteins.sum(axis=1).sort_values(ascending=False).index.values
# df_proteins = df_proteins.loc[protein_ids_ordered, :]
# Transport for figure
df_proteins = df_proteins.T
df_proteins

### Reordered

In [ ]:
# Order by number of proteins detected in a single study
df_proteins_reordered = df_proteins.T.copy()
pubmed_ids_ordered = (
    df_proteins_reordered.sum(axis=0).sort_values(ascending=False).index.values
)
df_proteins_reordered = df_proteins_reordered.loc[:, pubmed_ids_ordered]
df_proteins_reordered = df_proteins_reordered.T
df_proteins_reordered

### Visualization

In [ ]:
df_figure_data = df_proteins.copy()
df_figure_data = df_proteins_reordered.copy()
min_count = 0
max_count = 20
fontsize = 12

edgecolor = "black"
edgewidth = 0.5
bar_size = 0.75
as_percentages = True
cmin, cmax = (0.05, 1)

cmap = mpl.colormaps.get_cmap("terrain_r")
no_evidence_color = mpl.colors.to_rgba_array("xkcd:white")

In [ ]:
all_figure_data["ProteomicEvidenceAll"] = df_figure_data.copy()


df_value_counts = df_figure_data.sum(axis=0).value_counts().sort_index()
value_counts = df_value_counts[df_value_counts.index <= max_count].to_dict()
value_counts[max(value_counts.keys())] += df_value_counts[
    df_value_counts.index > max_count
].sum()
value_counts = {int(k): v for k, v in value_counts.items()}
df_value_counts = pd.Series(value_counts, name="Total")
id_fix_dict = {max_count: rf"$\geq${str(max_count)}"}
if min_count:
    value_counts = df_value_counts[df_value_counts.index >= min_count].to_dict()
    value_counts[min(value_counts.keys())] += df_value_counts[
        df_value_counts.index < min_count
    ].sum()
    value_counts = {int(k): v for k, v in value_counts.items()}
    id_fix_dict[min_count] = rf"{str(min_count)}$\geq$"

value_counts = {id_fix_dict.get(k, k): v for k, v in value_counts.items()}


df_value_counts = pd.DataFrame.from_dict(
    value_counts, orient="index", columns=["Total"]
)
df_value_counts.index.name = "Proteomic evidence (#studies)"
# Add colormap to table
cmap_dict = dict(
    zip(
        df_value_counts.index, cmap(np.linspace(cmin, cmax, len(df_value_counts.index)))
    )
)

df_value_counts_colors = df_value_counts.copy()
df_value_counts_colors["Colors"] = cmap_dict
df_value_counts_colors = df_value_counts_colors.reindex(
    index=df_value_counts_colors.index[::-1]
)
df_value_counts_colors.index = df_value_counts_colors.index.astype(str)
df_value_counts_colors

In [ ]:
orientation = "horizontal"
figsize = (1 if orientation == "vertical" else 6, 6 if orientation == "vertical" else 1)
fig_colorbar, cbar_ax = plt.subplots(figsize=figsize, layout="constrained")

cmap_dict = df_value_counts_colors["Colors"].to_dict()
cmap_segmented = mpl.colors.LinearSegmentedColormap.from_list(
    "legend", list(cmap_dict.values()), len(cmap_dict)
)
xticks = list(np.linspace(0, len(cmap_dict) - 1, len(cmap_dict)) + 0.5)
if orientation == "vertical":
    cmap_segmented = cmap_segmented.reversed("legend")
    xticks.reverse()

norm = mpl.colors.BoundaryNorm(
    np.linspace(0, len(cmap_dict), len(cmap_dict) + 1), cmap_segmented.N
)
colorbar = fig_colorbar.colorbar(
    mpl.cm.ScalarMappable(norm=norm, cmap=cmap_segmented),
    cax=cbar_ax,
    orientation=orientation,
    label=df_value_counts_colors.index.name,
)
colorbar.set_ticks(
    xticks, minor=False, labels=df_value_counts_colors.index, fontsize=fontsize - 2
)
colorbar.set_ticks([], minor=True)
cbar_ax.set_xlabel(df_value_counts_colors.index.name, fontdict={"size": fontsize})
fig_colorbar;

In [ ]:
fig = mpl.figure.Figure(figsize=(11, 6))
gs = mpl.gridspec.GridSpec(2, 1, hspace=0, height_ratios=[0.05, 1])

df_proteomic_cmap = df_figure_data.sum()
df_proteomic_cmap = df_proteomic_cmap.apply(lambda x: x if x < max_count else max_count)
df_proteomic_cmap = df_proteomic_cmap.apply(lambda x: x if x > min_count else min_count)

# Add col colors (similar to seaborn)
cmap_dict = df_value_counts_colors["Colors"].to_dict()
cmap_segmented = mpl.colors.LinearSegmentedColormap.from_list(
    "legend", [cmap_dict[c] for c in df_value_counts_colors.index[::-1]], len(cmap_dict)
)

ax_nstudies_colors = sns.heatmap(
    np.array([df_proteomic_cmap.values]),
    ax=fig.add_subplot(gs[0]),
    xticklabels=False,
    yticklabels=False,
    cmap=cmap_segmented,
    cbar=False,
)


ax_heatmap = sns.heatmap(
    df_figure_data.rename(
        {row: row.lstrip("pubmed:") for row in df_figure_data.index}, axis=0
    ),
    ax=fig.add_subplot(gs[1]),
    cbar=False,
    cmap=mpl.colors.ListedColormap(["xkcd:dried blood", "xkcd:red"]),
    xticklabels=False,
)
ax_heatmap.set_xlabel(
    f"Proteins (n = {len(df_figure_data.columns)})", fontsize=fontsize
)
ax_heatmap.set_ylabel(None)
ax_heatmap.yaxis.tick_left()
ax_heatmap.tick_params(axis="y", labelsize=fontsize)

fig.tight_layout()
if overwrite:
    fig.savefig(
        results_path / f"AllProteomicEvidence_PanelA.{imagetype}",
        transparent=transparent,
        format=imagetype,
    )
fig

In [ ]:
fig, ax_barchart = plt.subplots(1, 1, figsize=(7.5, 4))
sns.despine(ax=ax_barchart)

df_total_proteins = df_value_counts_colors["Total"]
rects = ax_barchart.bar(
    df_total_proteins.index,
    df_total_proteins.values,
    bar_size,
    color=df_value_counts_colors["Colors"].values,
    edgecolor=edgecolor,
    linewidth=edgewidth,
)
ax_barchart.bar_label(
    rects, labels=df_total_proteins.values, padding=1, fontsize=fontsize - 3
)
ax_barchart.set_xlabel(df_value_counts_colors.index.name, fontsize=fontsize)
ax_barchart.set_ylabel(f"Number of proteins", fontsize=fontsize)
all_figure_data["ProteomicEvidenceSummary"] = df_total_proteins.copy()

fig.tight_layout()
if overwrite:
    fig.savefig(
        results_path / f"AllProteomicEvidence_PanelB.{imagetype}",
        transparent=transparent,
        format=imagetype,
    )
fig;

### Aggregate Panels for visualization

In [ ]:
fig = mpl.figure.Figure(figsize=(12, 10))
gs = mpl.gridspec.GridSpec(2, 1, height_ratios=[1, 0.4])
sgs = mpl.gridspec.GridSpecFromSubplotSpec(
    2, 1, hspace=0, height_ratios=[0.05, 1], subplot_spec=gs[0]
)

has_additional_evidence_color, no_additional_evidence_color = (
    mpl.colors.to_rgba_array("xkcd:dark grey"),
    np.array([x / 2 + 0.5 for x in mpl.colors.to_rgba_array("xkcd:light grey")]),
)

df_proteomic_cmap = df_figure_data.sum()
df_proteomic_cmap = df_proteomic_cmap.apply(lambda x: x if x < max_count else max_count)
df_proteomic_cmap = np.array([df_proteomic_cmap.values])

# Add col colors (similar to seaborn)
cmap_dict = df_value_counts_colors["Colors"].to_dict()
cmap_segmented = mpl.colors.LinearSegmentedColormap.from_list(
    "legend", [cmap_dict[c] for c in df_value_counts_colors.index[::-1]], len(cmap_dict)
)

ax_nstudies_colors = sns.heatmap(
    df_proteomic_cmap,
    ax=fig.add_subplot(sgs[0]),
    xticklabels=False,
    yticklabels=False,
    cmap=cmap_segmented,
    cbar=False,
)

# Heatmap
ax_heatmap = sns.heatmap(
    df_figure_data.rename(
        {row: row.lstrip("pubmed:") for row in df_figure_data.index}, axis=0
    ),
    ax=fig.add_subplot(sgs[1]),
    cbar=False,
    cmap=mpl.colors.ListedColormap(["xkcd:dried blood", "xkcd:red"]),
    xticklabels=False,
)
ax_heatmap.set_xlabel(None)
ax_heatmap.set_ylabel(None)
ax_heatmap.yaxis.tick_left()
ax_heatmap.tick_params(axis="y", labelsize=fontsize)
ax_heatmap.set_xlabel(
    f"Proteins (n = {len(df_figure_data.columns)})", fontsize=fontsize
)

# Barchart
ax_barchart = fig.add_subplot(gs[1])
sns.despine(ax=ax_barchart)

df_total_proteins = df_value_counts_colors["Total"]
rects = ax_barchart.bar(
    df_total_proteins.index,
    df_total_proteins.values,
    bar_size,
    color=df_value_counts_colors["Colors"].values,
    edgecolor=edgecolor,
    linewidth=edgewidth,
)
ax_barchart.bar_label(
    rects, labels=df_total_proteins.values, padding=1, fontsize=fontsize - 3
)
ax_barchart.set_xlabel(df_value_counts_colors.index.name, fontsize=fontsize)
ax_barchart.set_ylabel(f"Number of proteins", fontsize=fontsize)
fig.tight_layout()
if overwrite:
    fig.savefig(
        results_path / f"AllProteomicEvidence_PanelAB.{imagetype}",
        transparent=transparent,
        format=imagetype,
    )
fig

## Export Figures and Data

In [ ]:
print(all_figure_data.keys())
if overwrite:
    with pd.ExcelWriter(results_path / "AllProteomicsData.xlsx") as writer:
        for sheet_name, df in all_figure_data.items():
            df = df.T.reset_index(drop=False)
            df.to_excel(writer, sheet_name=sheet_name, index=False)

#### Look up specific protein
Paste a list of UniProt Identifiers into the string to look up how many times those proteins were detected

In [ ]:
plist = """

""".strip().split(
    "\n"
)
df = df_figure_data.T
df = df[df.index.isin(plist)]

# df[df > 10]
print(df.T)
df = df.sum(axis=1)
# df = df[df <= 20]
# df = df[df >= 15]
for x in df.index:
    print(x)
df

#### Example
Example given is for removed proteins.

In [ ]:
plist = """
Q5T3U5
Q08828
O60266
Q8NFM4
O95622
O43306
P51828
P40145
O60503
O15120
Q9NUQ2
O94788
P17707
Q06278
Q9UN42
Q92903
Q9Y6K0
P35790
Q9Y259
Q8WUD6
Q05315
Q86VU5
P36551
P23786
Q9Y6T7
Q16760
P52429
P49619
Q86XP1
O75912
P52824
Q13574
P22413
Q9HBU6
Q9NVF9
P09467
O00757
P22830
P32189
Q14410
Q14409
Q9HCL2
Q86UL3
P09210
P33402
O75343
P25092
Q02846
P51841
P52789
Q2TB90
P09601
P30519
P20839
P23677
P27987
Q96DU7
Q9UHV8
Q00266
O95544
O75414
Q9Y5B8
Q9NWW6
P16066
P20594
O14841
Q9Y233
Q9HCR9
Q01064
Q14123
P27815
Q07343
Q08493
Q13946
Q9NP56
O60658
O95263
P16118
O60825
Q16875
Q16877
Q9UBF8
Q9Y2I7
O14986
P14555
O60733
O14494
O43688
O14495
P50336
P51606
Q9H310
O15244
O75751
Q14542
Q9BYW1
Q6PXP3
Q9NY64
Q9H2H9
Q96QD8
Q969I6
P13866
P53794
Q92911
Q9GZV3
Q9UN76
P52569
Q8WY07
Q9Y6L6
Q9NPD5
P17735
O60701
P22309
P22310
Q8TF68
""".strip().split(
    "\n"
)
df = df_figure_data.T
df = df[df.index.isin(plist)]
for x in df.index:
    print(x)
# df[df > 10]
df.sum(axis=1)